In [9]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from matplotlib import pyplot as plt
import pandas as pd

多个输入多个输出

In [10]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full) # 默认test_size = 0.25

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# 划分两个输入
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]

一般方法构建模型

In [ ]:
# 一般方法构建网络
input_A = keras.layers.Input(shape = [5], name = "wide_input")
input_B = keras.layers.Input(shape = [6], name = "deep_input")
hidden1 = keras.layers.Dense(30, activation = "relu")(input_B)
hidden2 = keras.layers.Dense(30, activation = "relu")(hidden1)
concat = keras.layers.concatenate([input_A, input_B])
output = keras.layers.Dense(1, name = "main_output")(concat)
aux_output = keras.layers.Dense(1, name = "aux_output")(hidden2)
model = keras.Model(inputs = [input_A, input_B], outputs = [output, aux_output])

调用子类API构建模型

In [11]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

In [12]:
model = WideAndDeepModel(units = 30, activation = "relu")
model.compile(loss = ["mse", "mse"], loss_weights = [0.9, 0.1], optimizer = keras.optimizers.SGD(learning_rate=1e-3))
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=10,
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))
total_loss, main_loss, aux_loss = model.evaluate([X_test_A, X_test_B], [y_test, y_test])

Epoch 1/10
 11/363 [..............................] - ETA: 1s - loss: 5.2714 - output_1_loss: 5.1547 - output_2_loss: 6.3219  

2021-11-23 21:29:35.007750: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


361/363 [============================>.] - ETA: 0s - loss: 2.5553 - output_1_loss: 2.2514 - output_2_loss: 5.2912

2021-11-23 21:29:36.882367: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - 2s 6ms/step - loss: 2.5487 - output_1_loss: 2.2446 - output_2_loss: 5.2856 - val_loss: 2.0697 - val_output_1_loss: 1.8376 - val_output_2_loss: 4.1589
Epoch 2/10
363/363 [==============================] - 2s 5ms/step - loss: 1.1269 - output_1_loss: 0.9305 - output_2_loss: 2.8949 - val_loss: 1.0738 - val_output_1_loss: 0.9289 - val_output_2_loss: 2.3779